In [56]:
# function to round decimals to  one tenth
def tenth_round(x):
    return round(x*10)/10

# State_dict is a dictionary of all possible states within the environment
# Based on fidelity set below, this equates to 10000 states
state_dict = {}
state_dict[0]=[0,0,0,0]
for i in range(0,10000):
    state_dict[i]=[0,0,0,0]

# Build the state dictionary.
counter = 0
i_div = 0
# 10 divisions between 0 and 1 in 0.1 increments (Cart position)
for i in range(10):
    if (i_div == 1.1):
        i_div = 0
    j_div = 0.1
    # 10 divisions between 0 and 1 in 0.1 increments (Cart velocity)
    for j in range(10):
        if (j_div == 1.1):
            j_div = 0
        k_div = 0.2
        # 10 divisions between 0 and 1 in 0.1 increments (pole angle)
        for k in range(10):
            if (k_div == 1.1):
                k_div = 0
            l_div = 0.3
            # 10 divisions between 0 and 1 in 0.1 increments (pole velocity)
            for l in range(10):
                if (l_div == 1.1):
                    l_div = 0
                state_dict[counter] = [tenth_round(i_div),tenth_round(j_div),tenth_round(k_div),tenth_round(l_div)]
                #print("State {}: {}".format(counter, state_dict[counter]))
                l_div += 0.1
                counter += 1
                    
            k_div += 0.1
            
        j_div += 0.1
        
    i_div += 0.1
    
# Method to Run a single episode of Cart Pole
# Sourced from http://kvfrans.com/simple-algoritms-for-solving-cartpole/
def run_episode(env, parameters):  
    observation = env.reset()
    #print("Observation: {}".format(observation))
    totalreward = 0
    for _ in range(200):
        action = 0 if np.matmul(parameters,observation) < 0 else 1
        #print("Action: {}".format(action))
        observation, reward, done, info = env.step(action)
        #env.render()
        #time.sleep(0.01)
        totalreward += reward
        if done:
            break
    return totalreward

# Method to remap numbers from one range to another
import math
def remap(x, in_min, in_max, out_min, out_max):    
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

# Method to find a state within the state_dict
# Assumes that incoming state param contains values not formatted
# Maps state values to appropriate ranges and performs linear search    
def find_state(state):
    new_state = state
    new_state[0] = tenth_round(remap(state[0], -2.4,2.4,0,1))
    new_state[1] = tenth_round(remap(state[1], -3.4*10**38, 3.4*10**38, 0, 1))
    new_state[2] = tenth_round(remap(state[2], -0.5, 0.5, 0, 1))
    new_state[3] = tenth_round(remap(state[3], -3.4*10**38,3.4*10**38,0,1))
    
    for i in range (10000):
        if(state_dict[i][0] == new_state[0]):
            if(state_dict[i][1] == new_state[1]):
                if(state_dict[i][2] == new_state[2]):
                    if(state_dict[i][3] == new_state[3]):
                        return i
    # Returning -1 will flag that state was not found
    return -1

0: [0.0, 0.1, 0.2, 0.3]
1: [0.0, 0.1, 0.2, 0.4]
2: [0.0, 0.1, 0.2, 0.5]
3: [0.0, 0.1, 0.2, 0.6]
4: [0.0, 0.1, 0.2, 0.7]
5: [0.0, 0.1, 0.2, 0.8]
6: [0.0, 0.1, 0.2, 0.9]
7: [0.0, 0.1, 0.2, 1.0]
8: [0.0, 0.1, 0.2, 1.1]
9: [0.0, 0.1, 0.2, 1.2]
10: [0.0, 0.1, 0.3, 0.3]
11: [0.0, 0.1, 0.3, 0.4]
12: [0.0, 0.1, 0.3, 0.5]
13: [0.0, 0.1, 0.3, 0.6]
14: [0.0, 0.1, 0.3, 0.7]
15: [0.0, 0.1, 0.3, 0.8]
16: [0.0, 0.1, 0.3, 0.9]
17: [0.0, 0.1, 0.3, 1.0]
18: [0.0, 0.1, 0.3, 1.1]
19: [0.0, 0.1, 0.3, 1.2]
20: [0.0, 0.1, 0.4, 0.3]
21: [0.0, 0.1, 0.4, 0.4]


In [11]:
# Method to run a random episode
# Sourced and modified from http://kvfrans.com/simple-algoritms-for-solving-cartpole/
def run_random_episode(env, parameters): 
    epochs = 0
    penalties = 0
    observation = env.reset()
    totalreward = 0
    for _ in range(200):
        action = 0 if np.matmul(parameters,observation) < 0 else 1
        observation, reward, done, info = env.step(action)        
        epochs += 1
        totalreward += reward
        if done:
            break
    return totalreward, epochs, penalties

import gym
import numpy as np
import time

env = gym.make("CartPole-v0").env
total_epochs, total_penalties, total_rewards = 0, 0, 0

# Random search ------------------------------------------
episodes = 10

# Perform a random policy for x episodes
for _ in range(episodes):
    state = env.reset()
    new_state = find_state(state)
    epochs, penalties, reward = 0, 0, 0
    
    #Random Search
    bestparams = None  
    bestreward = 0  
    counter = 0;
    for _ in range(1):  
        parameters = np.random.rand(4) * 2 - 1
        reward, epochs, penalties = run_random_episode(env,parameters)
        penalties = 200 - reward
        total_penalties += penalties
        total_rewards += reward
        total_epochs += epochs
        if reward > bestreward:
            bestreward = reward
            bestparams = parameters
            # considered solved if the agent lasts 200 timesteps
            if reward == 200:
                break
        counter += 1
    print("Timesteps lasted: {}".format(epochs))

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Timesteps lasted: 91
Timesteps lasted: 149
Timesteps lasted: 11
Timesteps lasted: 15
Timesteps lasted: 9
Timesteps lasted: 9
Timesteps lasted: 10
Timesteps lasted: 8
Timesteps lasted: 9
Timesteps lasted: 11
Results after 10 episodes:
Average timesteps per episode: 32.2


In [63]:
%%time

import gym
import numpy as np
import time

env = gym.make("CartPole-v0")

#Q Learning -------------------------------------------
import random
from IPython.display import clear_output

#10000 x 2 table for storing Q Values
q_table = np.zeros([10000, 2])

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.3

# Perform agent learning for 1000 episodes. Currently executes in 30 seconds
for i in range(1000):
    # Get a new state, then find it in state_dict
    state = env.reset()
    current_state = find_state(state)
    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:       
        if random.uniform(0, 1) < epsilon:
            action_index = np.random.randint(2, size=1) # Explore action space
            action_index = int(action_index)
        else:
            action_index = np.argmax(q_table[current_state]) # Exploit learned values
    
        next_state, reward, done, info = env.step(action_index) 
        future_state = find_state(next_state)
        old_value = q_table[current_state, action_index]
        future_max = np.max(q_table[future_state])
        
        # Q Learning Algorithm
        new_value = old_value + alpha * (reward + (gamma * future_max) - old_value)
        
        q_table[current_state][action_index] = new_value

        current_state = future_state
        epochs += 1
           
    if i % 10 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")      

print("Training finished.\n")

Episode: 990
Training finished.

Wall time: 36.5 s


In [64]:
# Debuggin - check how many rows of Q Table were updated
counter = 0
for i in range(10000):
    if(q_table[i][0] != 0 or q_table[i][1] != 0):
        counter += 1
        #print("Q-table[{}]: {} -> state_dict: {}".format(i, q_table[i], state_dict[i]))
print("Rows filled: {}".format(counter))

Rows filled: 16


In [65]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties, total_rewards = 0, 0, 0

episodes = 10

# For x episodes run the program, always selecting the Max element from the Q Table
for _ in range(episodes):
    state = env.reset()
    new_state = find_state(state)
    epochs, penalties, reward = 0, 0, 0
    episode_reward = 0
    done = False
    
    for i in range(200):
        action = np.argmax(q_table[new_state])
        state, reward, done, info = env.step(action)
        episode_reward += reward       
        epochs += 1
        if done:
            break
            
    total_penalties += (200 - episode_reward)        

    total_epochs += epochs
    total_rewards += episode_reward

    print("Timesteps lasted: {}".format(epochs))

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")

Timesteps lasted: 10
Timesteps lasted: 10
Timesteps lasted: 10
Timesteps lasted: 9
Timesteps lasted: 9
Timesteps lasted: 9
Timesteps lasted: 9
Timesteps lasted: 9
Timesteps lasted: 9
Timesteps lasted: 10
Results after 10 episodes:
Average timesteps per episode: 9.4
